In [3]:

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import keras
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import matplotlib.pyplot as plt

In [2]:
data_args = dict(rescale=1./255, validation_split=.20)

In [5]:
img_dir = ('./dataset')

In [6]:
bag_train = tf.keras.preprocessing.image.ImageDataGenerator(**data_args)

train_gen = bag_train.flow_from_directory(
img_dir,
subset="training",
shuffle=True,
target_size=(224, 224))


bag_val = tf.keras.preprocessing.image.ImageDataGenerator(**data_args)

val_gen = bag_val.flow_from_directory(
img_dir,
subset="validation",
shuffle=True,
target_size=(224, 224))

Found 2352 images belonging to 2 classes.
Found 588 images belonging to 2 classes.


In [8]:
for image_batch, label_batch in train_gen:
    break
image_batch.shape, label_batch.shape

((32, 224, 224, 3), (32, 2))

In [9]:
print (train_gen.class_indices)

{'Autistic': 0, 'Non_Autistic': 1}


In [10]:
base_model = tf.keras.Sequential([
 hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", 
 output_shape=[1280],
 trainable=False),
 tf.keras.layers.Dropout(0.4),
 tf.keras.layers.Dense(train_gen.num_classes, activation='softmax')
])
base_model.build([None, 224, 224, 3])
base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [11]:
optimizer = tf.keras.optimizers.Adam(lr=0.001)

base_model.compile(
 optimizer=optimizer,
 loss= 'categorical_crossentropy',
 metrics=['accuracy'])

C:\Users\theja\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [37]:
import math
def calculate_spe(y):
  return int(math.ceil((1. * y) / 32))
steps_per_epoch = calculate_spe(2800)
steps_per_epoch 

88

In [ ]:

epochs=20
history = base_model.fit(
  train_gen,
  validation_data = val_gen,
  epochs = epochs
)

Epoch 1/20
74/74 [==============================] - 52s 698ms/step - loss: 0.3451 - accuracy: 0.8427 - val_loss: 0.6573 - val_accuracy: 0.7126
Epoch 2/20
74/74 [==============================] - 49s 666ms/step - loss: 0.3386 - accuracy: 0.8486 - val_loss: 0.6068 - val_accuracy: 0.7194
Epoch 3/20
74/74 [==============================] - 54s 725ms/step - loss: 0.3543 - accuracy: 0.8423 - val_loss: 0.5957 - val_accuracy: 0.7211
Epoch 4/20
74/74 [==============================] - 54s 724ms/step - loss: 0.3484 - accuracy: 0.8495 - val_loss: 0.6223 - val_accuracy: 0.7313
Epoch 5/20
74/74 [==============================] - 56s 758ms/step - loss: 0.3476 - accuracy: 0.8389 - val_loss: 0.6200 - val_accuracy: 0.7313
Epoch 6/20
74/74 [==============================] - 53s 724ms/step - loss: 0.3425 - accuracy: 0.8512 - val_loss: 0.5993 - val_accuracy: 0.7245
Epoch 7/20
74/74 [==============================] - 54s 729ms/step - loss: 0.3512 - accuracy: 0.8380 - val_loss: 0.6068 - val_accuracy: 0.7058

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
base_model.save('base_dir.h5')